In [261]:
%cd /home/ec2-user/SageMaker/srihari
from model_process.model import sop,model_TT,model_table
import numpy as np

/home/ec2-user/SageMaker/srihari


In [3]:
# pip install fpdf

In [4]:
# from fpdf import FPDF
# pdf = FPDF()
# # imagelist is the list with all image filenames
# for image in imagelist:
#     pdf.add_page()
#     pdf.image(image,x,y,w,h)
# pdf.output("yourfile.pdf", "F")

In [344]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import layoutparser as sop
from model_process.model import sop,model_TT,model_table
import base64


def extract_line(df_ocr, df_area):
    data=[]
    bold=[]
    for idn,model_row in df_area.iterrows():
#         tp_bbx=[0,int(model_row.y_1),int(model_row.page_width),int(model_row.y_2)] #andover
        tp_bbx=[int(model_row.x_1),int(model_row.y_1),int(abs(model_row.page_width-model_row.x_1)),int(model_row.y_2)]
        words= []
        is_bold = []
        for idx,ocr_row in df_ocr.iloc[:,:4].iterrows():
            word_bbx=ocr_row.astype(int).values
            flag=check_point(tp_bbx,word_bbx)
            if flag:
                words.append(df_ocr.iloc[idx]['text'].strip())
                is_bold.append(df_ocr.iloc[idx]['is_bold'])
        bold.append(is_bold)
        data.append(words)
    return data,bold


def check_title(df):
    
    final=[]
    for i,row in df.iterrows():
        if row.type in ['Title','text']:
            if (row.is_upper==True) and (row.word_length<5): # and (row.is_bold==True):
                final.append("Title")
            else:
                final.append("subtitle")
        else:
             final.append(row.type)
    df["final_type"] = final
    return df

def is_header_footer(ocr_df):
    ocr_df["combine_info"]= ocr_df[['x_1','y_1','x_2','y_2','text','height','width']].apply(lambda x: ','.join([str(p) for p in x]) ,axis=1)
    ocr_df["is_header_footer"]=ocr_df.combine_info.duplicated(keep='last')
    return ocr_df


def is_bulit(ocr_df):
    title_index=["1.0",'2.0','3.0','4.0','5.0','6.0','7.0','8.0','9.0','10.0']
    ocr_df['is_start_bulit']=ocr_df.text.apply(lambda x : x in title_index)
    return ocr_df

# word
def get_center(x1,y1,x2,y2):
    xCenter = (x1 + x2) / 2
    yCenter = (y1 + y2) / 2
    return xCenter,yCenter

def FindPoint(x1, y1, x2,
              y2, x, y) :
    if (x > x1 and x < x2 and
        y > y1 and y < y2) :
        return True
    else :
        return False

def check_point(tt_box,word_bbx):
    point_x,point_y = get_center(word_bbx[0],word_bbx[1],
                                 word_bbx[2],word_bbx[3])
    return FindPoint(tt_box[0],tt_box[1],
                     tt_box[2],tt_box[3],point_x,point_y)

def extract_words(df_ocr, page_index, layout_df, page_img):
    df_ocr=df_ocr[df_ocr.page_index==page_index]
    df_area = layout_df.sort_values(by=['y_1'], ascending=True)
    data=[]
    for idn,model_row in df_area.iterrows():
        para_data = {}
        para_data["page_index"]=int(model_row.page_index)
        para_data["type"]=model_row.type
        para_data["final_type"]=model_row.final_type.lower()
        if model_row.final_type.lower() == "table":
            img_cp=cropimage(page_img.copy(), int(model_row.x_1), int(model_row.y_1), int(model_row.x_2),int(model_row.y_2))
            para_data["image"]=None
            if img_cp is not None:
                para_data["image"] = numpy_base64(img_cp)
#         tp_bbx=[0,int(model_row.y_1),int(model_row.page_width),int(model_row.y_2)] #model_row[:4].astype(int).values andover
        tp_bbx=[int(model_row.x_1),int(model_row.y_1),int(abs(model_row.page_width-model_row.x_1)),int(model_row.y_2)]
        word_index=[]
        for idx,ocr_row in df_ocr.iloc[:,:4].iterrows():
            word_bbx=ocr_row.astype(int).values
            flag=check_point(tp_bbx,word_bbx)
            if flag:
                word_index.append(idx)
        
        content=" ".join([df_ocr.iloc[x]['text'] for x in word_index])
        if content.lower().startswith("note"):
            para_data["final_type"]="note"           
        para_data['text']=content
        data.append(para_data)
    
    return data

def mask_image(df,page_image):
    for x in df.iloc[:,:4].astype(int).values:
        page_image = cv2.rectangle(page_image, tuple(x[:2]),tuple(x[2:]),[255,255,255],-1)
    return page_image

def clean_image(image, ocr_df, page_index):
    page_id=page_index
    ocr_df=is_bulit(ocr_df)
    ocr_df=is_header_footer(ocr_df)
    select_page = ocr_df[ocr_df.page_index==page_id]
    remove_header_footer = select_page[select_page.is_header_footer==True]
    remove_bulit = select_page[select_page.is_start_bulit==True]
    
    image_header=mask_image(remove_header_footer, image.copy())
    _image = mask_image(remove_bulit, image_header)
    
    return _image


def correct_df_area(page_ocr,page_index,page_area):
    page_ocr=page_ocr[page_ocr.page_index==page_index]
    sent,bold=extract_line(page_ocr,page_area)
    page_area["data"]=sent
    page_area["is_bold"]=[all(list_bold) for list_bold in bold]
    page_area["clean_data"]=[" ".join(x) for x in sent]
    page_area['word_length']=page_area.data.apply(lambda x: len(x))
    page_area['is_upper']=page_area.clean_data.apply(lambda x: x.isupper())
    update_df=check_title(page_area)
    update_df["is_duplicate"]=update_df.clean_data.duplicated()
    
    return update_df[update_df.is_duplicate==False]


def load_ocr(filename=None):
    pdf_layout, pdf_images = sop.load_pdf(filename,load_images=True)
    ocr_data=[]
    for page_index,ocr in enumerate(pdf_layout):
        image=np.array(pdf_images[page_index])
        df=ocr.to_dataframe()
        df["page_index"]=page_index
        df["height"] = abs(df.y_1-df.y_2)
        df['width'] = abs(df.x_1-df.x_2)
        df['page_width'] = image.shape[1]
        df['page_height'] = image.shape[0]
        df['clen_type']=df.type.apply(lambda x: x.lower())
        df['is_bold']=df.clen_type.str.contains('bold')
        ocr_data.append(df)
    ocr_df=pd.concat([x for x in ocr_data])
    return ocr_df,pdf_images
    

def layout_model(image,page_index=0, table=True):
    temp_df = pd.DataFrame({'x_1':[], 'y_1':[], 'x_2':[], 'y_2':[],
                            'block_type':[], 'type':[], 'score':[], 'page_index':[],
                            'height':[], 'width':[], 'page_width':[], 'page_height':[]})
    if table:
        table_layout=model_table.detect(image)
        image=mask_image(table_layout.to_dataframe(),image)
        table_layout= table_layout.to_dataframe()
    layout=model_TT.detect(image)
    df=layout.to_dataframe()
    df = pd.concat([temp_df,df,table_layout])
    df["page_index"] = page_index
    df["height"] = abs(df.y_1-df.y_2)
    df['width'] = abs(df.x_1-df.x_2)
    df['page_width'] = image.shape[1]
    df['page_height'] = image.shape[0]
    df = df.sort_values(by=['y_1'], ascending=True)
    return df

def line_by_first_bbx(line_df,ocr_df,hight=0):
    tp_bbx=[0,int(line_df.y_1-hight),int(line_df.page_width),int(line_df.y_2+hight)]
    words= []
    for idx,ocr_row in ocr_df.iloc[:,:4].iterrows():
        word_bbx=ocr_row.astype(int).values
        flag=check_point(tp_bbx,word_bbx)
        if flag:
            words.append(idx)
    _line=ocr_df.loc[words]
    return _line.sort_values(by=['x_1'], ascending=True)

def Convert(a):
    it = iter(a)
    res_dct = dict(zip(it, it))
    return res_dct

def header_extraction(ocr_df):
    try:
        ocr_df=is_header_footer(ocr_df)
        header=ocr_df[(ocr_df.is_header_footer==True) & (ocr_df.page_index==2)]
        header[["x_1","y_1","x_2","y_2"]]=header[["x_1","y_1","x_2","y_2"]].astype(int)

        header=ocr_df[(ocr_df.is_header_footer==True) & (ocr_df.page_index==2)]
        header[["x_1","y_1","x_2","y_2"]]=header[["x_1","y_1","x_2","y_2"]].astype(int)
        line1=header[(header.text=="Status") & (header.is_bold==True)]
        line2=header[(header.text=="Title") & (header.is_bold==True)]
        first_line=line_by_first_bbx(line1,header)
        title=line_by_first_bbx(line2,header,abs(line2.y_1-line2.y_2))

        header_list={"Status":"Status","Effective Date":"Effective-Date","Version":"Version","Doc Name":"Doc-Name"}

        raw=" ".join(first_line.text)
        for k,v in header_list.items():
            raw=raw.replace(k,v)
        header_dict=Convert(raw.split())
        title_text=" ".join(title.sort_index().text).replace('Title','').strip()
        header_dict["Title"]=title_text
        return header_dict
    except:
        return {}

def kmz_heading(ocr_df):
    title_index=['I.','XI.','II.' 'XII.','III.','XIII.','IV.','XIV.','V.','XV.','VI.','XVI.','VII.','XVII.','VIII.','XVIII.','IX.','XIX.','X.','XX.']
    ocr_df['kmz_heading']=ocr_df.apply(lambda x : x.text in title_index and x.is_bold==True, axis=1)
    return ocr_df

def kmz_title_hur(ocr):
    ocr=kmz_heading(ocr)
    title_df=ocr[ocr.kmz_heading==True]
    title_df=title_df[['x_1','y_1','x_2','y_2','page_index','page_width','page_height']]
    title_df['block_type']='rectangle'
    title_df['type']='Title'
    title_df['score']=-1
    return title_df

def doc_type(ocr):
    df_page1=ocr[ocr.page_index==0]
    if len(df_page1[df_page1.text=='Kalamazoo'])>0:
        return "kmz"
    else:
        return "andover"

def cropimage(image,x1,y1,x2,y2):
    try:
        cropped_image = image[y1:y2, x1:x2]
    except:
        print("image was empty for table")
        return None
    return cropped_image
 
def numpy_base64(cropped_image):
    _,imarry=cv2.imencode('.jpg',img_cp)
    inbyte=imarry.tobytes()
    bs4=base64.b64encode(inbyte)
    return bs4

In [335]:
ocr, pdf_images = load_ocr("data/sop/MCD-024737.pdf")

# ocr=kmz_title_hur(ocr)

In [71]:
# layout=model_table.detect(img)
# layout1=model_TT.detect(img)

In [284]:
# layout.to_dataframe()

In [283]:
# ocr

In [336]:

page_index=2
image_org=np.asarray(pdf_images[page_index])
image=clean_image(image_org.copy(),ocr,page_index) #remove header
df_title=kmz_title_hur(ocr) #find title kmz
image=mask_image(ocr[ocr.kmz_heading==True],image) #remove roman




In [337]:
# def layout_model(image,page_index=0, table=True):

type_ann=["Text","List"]
threshold=0.70
temp_df = pd.DataFrame({'x_1':[], 'y_1':[], 'x_2':[], 'y_2':[],
                        'block_type':[], 'type':[], 'score':[], 'page_index':[],
                        'height':[], 'width':[], 'page_width':[], 'page_height':[]})

table_layout=model_table.detect(image)
image_mask=mask_image(table_layout.to_dataframe(),image)
table_layout= table_layout.to_dataframe()
layout=model_TT.detect(image_mask)
df_=layout.to_dataframe()
df_tt=df_[df_.type.apply(lambda x: x in type_ann)] ## text and list from model tt pre-train
df_tt = df_tt[df_tt.score>=threshold] #only for tt model
title_hur=df_title[df_title.page_index==page_index]
df = pd.concat([temp_df,df_tt,table_layout,title_hur])

df["page_index"] = page_index
df["height"] = abs(df.y_1-df.y_2)
df['width'] = abs(df.x_1-df.x_2)
df['page_width'] = image.shape[1]
df['page_height'] = image.shape[0]
df.reset_index(inplace=True)
df = df.sort_values(by=['y_1'], ascending=True)


In [339]:
layout_df=correct_df_area(ocr[ocr.page_index==page_index],page_index,df)
layout_df

,index,x_1,y_1,x_2,y_2,block_type,type,score,page_index,height,width,page_width,page_height,data,is_bold,clean_data,word_length,is_upper,final_type,is_duplicate
2,51,43.440000,168.792000,59.756000,180.792000,rectangle,Title,-1.000000,2,12.000000,16.316000,612,792,"[IV., REAGENT, LIST]",True,IV. REAGENT LIST,3,True,Title,False
1,1,41.635933,178.751190,577.861328,293.448029,rectangle,Table,0.882884,2,114.696838,536.225395,612,792,"[Reagents, Component, Criteria, Water, HPLC, g...",False,Reagents Component Criteria Water HPLC grade S...,43,False,Table,False
3,97,48.120000,303.912000,59.758000,315.912000,rectangle,Title,-1.000000,2,12.000000,11.638000,612,792,"[V., SOLUTION, PREPARATION]",True,V. SOLUTION PREPARATION,3,True,Title,False
0,0,34.141487,317.016693,586.352905,593.174072,rectangle,Table,0.899820,2,276.157379,552.211418,612,792,"[Mobile, Phase, Notebook, #, Expiry, May, be, ...",False,Mobile Phase Notebook # Expiry May be scaled u...,88,False,Table,False


In [345]:
data = extract_words(ocr_df,page_index,layout_df,image_org)

In [ ]:
table_layout=model_Ta.detect(image)
sop.draw_box(image_org,table_layout,show_element_type=True)   

In [6]:
layout=model_TT.detect(image)
layout_table=model_table.detect(image)

/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/detectron2/modeling/roi_heads/fast_rcnn.py:154: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:882.)
  filter_inds = filter_mask.nonzero()


In [17]:
pdf_layout[0].to_dataframe()

,x_1,y_1,x_2,y_2,block_type,text,id,type
0,207.120,45.192,232.366,57.192,rectangle,FOR,0,"XMIDWP+TimesNewRoman,Bold"
1,235.440,45.192,279.414,57.192,rectangle,PFIZER,1,"XMIDWP+TimesNewRoman,Bold"
2,282.402,45.192,345.801,57.192,rectangle,INTERNAL,2,"XMIDWP+TimesNewRoman,Bold"
3,348.831,45.192,372.175,57.192,rectangle,USE,3,"XMIDWP+TimesNewRoman,Bold"
4,375.213,45.192,409.938,57.192,rectangle,ONLY,4,"XMIDWP+TimesNewRoman,Bold"
...,...,...,...,...,...,...,...,...
290,219.000,777.940,237.000,787.940,rectangle,On:,290,Courier
291,243.000,777.940,309.000,787.940,rectangle,13-Jul-2022,291,Courier
292,315.000,777.940,369.000,787.940,rectangle,"09:21:14,",292,Courier
293,375.000,777.940,393.000,787.940,rectangle,GMT,293,Courier


In [ ]:
sop.draw_box(img, layout,
            box_width=3,
            show_element_id=True,show_element_type=True)

In [79]:
# p=pd.DataFrame({"A":[1,2,3,1,1,2,1]})

In [124]:
# p["is_dup"]=p.duplicated(['A'],keep=False)